In [55]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import seaborn as sb
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from collections import Counter
import scipy.stats as ss
from tensorflow import keras
import tensorflow as tf
from sklearn import preprocessing,model_selection


In [130]:
df_train = pd.read_csv("train.tsv", delimiter='\t')
df_test = pd.read_csv("test.tsv", delimiter='\t')

In [131]:
df_train=df_train.drop(['train_id'],1)
df_test=df_test.drop(['test_id'],1)

In [132]:
df_train['category1'], df_train['category2'],df_train['category3'] = df_train['category_name'].str.split('/',2).str

In [133]:
trainData=df_train.copy()
le = preprocessing.LabelEncoder()
trainData['category_id']= le.fit_transform(df_train.category_name.astype(str))
trainData['brand_id']= le.fit_transform(df_train.brand_name.astype(str))
trainData['category1Id']= le.fit_transform(df_train.category1.astype(str))
trainData['category2Id']= le.fit_transform(df_train.category2.astype(str))
trainData['category3Id']= le.fit_transform(df_train.category3.astype(str))

In [6]:
#trainData['logPrice']=np.log(trainData['price']+1)

In [134]:
X = trainData[['item_condition_id', 'shipping', 'category_id', 'brand_id', 'category1Id','category2Id','category3Id']]
y = trainData['price']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y,test_size= 0.2)

In [135]:
min_max_scaler = preprocessing.MinMaxScaler()

np_scaled = min_max_scaler.fit_transform(X_train.astype(float))
X_train_normalized = pd.DataFrame(np_scaled)

np_scaled = min_max_scaler.fit_transform(X_test.astype(float))
X_test_normalized = pd.DataFrame(np_scaled)

In [136]:
y_train = np.log1p(y_train)
y_test = np.log1p(y_test)

In [137]:
clfReg= LinearRegression().fit(X_train, y_train)

In [138]:
#returns R2
print('Logistic Regression Accuracy',clfReg.score(X_test, y_test))

Logistic Regression Accuracy 0.08439888312160038


In [139]:
X_train.dtypes

item_condition_id    int64
shipping             int64
category_id          int64
brand_id             int64
category1Id          int64
category2Id          int64
category3Id          int64
dtype: object

In [140]:
X_train_normalized = X_train_normalized.as_matrix()
y_train = y_train.as_matrix()

X_test_normalized = X_test_normalized.as_matrix()
y_test = y_test.as_matrix()

In [141]:
#buidling NN
model = keras.Sequential([
    keras.layers.Dense(8, activation=tf.nn.relu),
    keras.layers.Dense(1024, activation=tf.nn.relu),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dense(256),
    keras.layers.Dense(128),
    keras.layers.Dense(64),
    keras.layers.Dense(1)
    ])

model.compile(optimizer=tf.train.AdadeltaOptimizer(),
              loss ='mean_squared_error',
              )
model.fit(X_train_normalized, y_train, batch_size= 2000, epochs=10)

Epoch 1/10
1186028/1186028 [==============================] - 74s 62us/step - loss: 8.4866
Epoch 2/10
1186028/1186028 [==============================] - 73s 61us/step - loss: 2.5114
Epoch 3/10
1186028/1186028 [==============================] - 69s 58us/step - loss: 0.9972
Epoch 4/10
1186028/1186028 [==============================] - 73s 61us/step - loss: 0.9338
Epoch 5/10
1186028/1186028 [==============================] - 75s 63us/step - loss: 0.8817
Epoch 6/10
1186028/1186028 [==============================] - 69s 58us/step - loss: 0.8369
Epoch 7/10
1186028/1186028 [==============================] - 71s 60us/step - loss: 0.7975
Epoch 8/10
1186028/1186028 [==============================] - 73s 62us/step - loss: 0.7633
Epoch 9/10
1186028/1186028 [==============================] - 72s 60us/step - loss: 0.7308
Epoch 10/10
1186028/1186028 [==============================] - 72s 61us/step - loss: 0.7003


In [144]:
test_loss = model.evaluate(X_test_normalized, y_test)

296507/296507 [==============================] - 47s 159us/step


In [145]:
test_loss

0.683937259670162

In [148]:
y_predict=model.predict(X_test_normalized)

In [149]:
y_predict

array([[3.3699567],
       [3.5903122],
       [3.3631659],
       ...,
       [3.2445807],
       [3.4302447],
       [3.4261034]], dtype=float32)

In [154]:
y_predict =np.expm1(y_predict)

In [155]:
y_test = np.expm1(y_test)

In [156]:
mse = mean_squared_error(y_predict, y_test)

In [157]:
mse

1526.1518565810866